In [1]:
import numpy as np
from logistic_regression.GrdDscntQuant import grdescentquant
from logistic_regression.normal_logistic import normallogistic
from logistic_regression.GrdDscnt import grdescentnormal
from logistic_regression.quant_logistic import quant_logistic
from logistic_regression.GrdDscntUncoded import grdescentuncoded
from logistic_regression.uncoded_logistic import  uncoded_logistic
from quantization.quantize import quantize
from coded_computation.master import master
import matplotlib.pyplot as plt
import pandas as pd
from scipy.io import arff
from pad_and_clean import pad
from pad_and_clean import clean_and_scale
import time
from gen_data import gen_data, gen_nonlinear_data
from sklearn.model_selection import train_test_split

def get_loss(w,X,y):
    #calculates 1-0 prediction error
    log_odds = X@w
    probs = 1 / (1 + np.exp(-log_odds))
    preds = np.where(probs > 0.5, 1,-1)
    test_loss = np.mean(preds != y)

    return test_loss
def plot_3d_bar(data, z):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')

    # Define the x, y coordinates and the z heights
    _x = np.arange(data.shape[1])
    _y = np.arange(data.shape[0])
    _x, _y = np.meshgrid(_x, _y)
    x, y = _x.ravel(), _y.ravel()

    # The z values represent the bar heights
    z = np.zeros_like(x)
    dz = data.ravel()

    # Plot 3D bars
    ax.bar3d(x, y, z, 1, 1, dz, shade=True)

    # Labels
    ax.set_xlabel('w_lvl')
    ax.set_ylabel('grd_lvl')
    ax.set_zlabel(z)

    plt.show()

def run(X,y, filename):
    func = quant_logistic
    G = np.array([
        [1, 1, 1, 1, 1, 1, 1],
        [-1, -1, -1, 1, 1, 1, 1],
        [-1, 1, 1, -1, -1, 1, 1],
        [1, -1, -1, -1, -1, 1, 1],
        [1, -1, 1, -1, 1, -1, 1],
        [-1, 1, -1, -1, 1, -1, 1],
        [-1, -1, 1, 1, -1, -1, 1],
        [1, 1, -1, 1, -1, -1, 1]
    ]).T

    stepsize = 0.1
    maxiter = 10000

    #X,y = gen_nonlinear_data(500, 40, 1)

    X, Xt, y, yt = train_test_split(X, y, test_size=0.2)
    X, y = pad(X, y, 7)
    Xt, yt = pad(Xt, yt, 7)

    Master_uncoded = master(X, None, 21)
    Master = master(X, G, 3)
    times_grid = np.zeros((5, 4))
    loss_grid = np.zeros((5, 4))  # Initialize for each dataset
    iters_grid = np.zeros((5, 4))
    test_loss = np.zeros((5, 4))
    test_loss_uncoded = np.zeros((5,4))
    uncoded_times = np.zeros((5, 4))
    # loss from normal logistic regression
    w0 = np.random.uniform(-1, 1, (X.shape[1], 1))
    start_time = time.time()
    w, num_iters = grdescentnormal(normallogistic, w0, stepsize, maxiter, Master_uncoded, y, X, tolerance=1e-02)
    normal_loss = get_loss(w, Xt, yt)
    end_time = time.time()
    print(f"loss from unquantized logistic regresison: {normal_loss} on {num_iters} iterations in {end_time - start_time} seconds")
    repetitions = 100
    for i in range(repetitions):
        for w_lvl in range(4, 9):
            for grd_lvl in range(2, 6):
                # logic for calculations
                w0 = np.random.uniform(-1, 1, (X.shape[1], 1))

                start_time = time.time()
                #w, num_iters = grdescentquant(func, w0, stepsize, maxiter, Master, w_lvl, grd_lvl, X, y, filename, tolerance=1e-02)
                w, num_iters = grdescentquant(func, w0, stepsize, maxiter, Master, w_lvl, grd_lvl, X, y, filename, 1e-02)
                # grdescentquant(func, w, stepsize, maxiter, Master, w_lvl, grd_lvl, X, y, filename, tolerance)

                end_time = time.time()

                times_grid[w_lvl - 4, grd_lvl - 2] += end_time - start_time
                loss_grid[w_lvl - 4, grd_lvl - 2] += get_loss(w, X, y)
                iters_grid[w_lvl - 4, grd_lvl - 2] += num_iters
                test_loss[w_lvl - 4, grd_lvl - 2] += get_loss(w, Xt, yt)

                start_time = time.time()
                w, num_iters = grdescentuncoded(uncoded_logistic, w0, stepsize, maxiter, Master_uncoded, w_lvl, grd_lvl, X, y, tolerance=1e-02)
                end_time = time.time()
                uncoded_times[w_lvl - 4, grd_lvl - 2] += end_time - start_time
                test_loss_uncoded[w_lvl - 4, grd_lvl - 2] += get_loss(w,Xt,yt)

    test_loss = test_loss/repetitions
    loss_grid = loss_grid/repetitions
    iters_grid = iters_grid/repetitions
    times_grid = times_grid/repetitions
    test_loss_uncoded = test_loss_uncoded/repetitions
    uncoded_times = uncoded_times/repetitions



    print(f" training loss: \n{loss_grid}")
    print(f" avg iterations: \n{iters_grid}")
    print(f" test loss: \n{test_loss}")
    print(f"run time: \n{times_grid}")
    print(f" test loss uncoded: \n{test_loss_uncoded}")
    print(f" times uncoded: \n{uncoded_times}")
    return test_loss, loss_grid, iters_grid, times_grid, test_loss_uncoded, uncoded_times

(500, 45) (500, 1)
(400, 45) (400, 1)
(100, 45) (100, 1)


In [2]:
X, y = gen_nonlinear_data(500, 40, 1)

test_loss, loss_grid, iters_grid, times_grid, test_loss_uncoded, uncoded_times = run(X,y, "nonlinear_data.csv")

loss from unquantized logistic regresison: 0.37142857142857144 on 394 iterations in 0.08082318305969238 seconds
response, actual 
 [[27.71681189  1.46413096]
 [19.80796433  2.46779736]
 [-0.0863594  -1.70987823]
 [ 5.82709958 -1.47425213]
 [47.02534799  3.24531103]] 

index passed: [[-0.37851602]
 [-0.37542618]
 [-0.37233634]
 [-0.3692465 ]
 [-0.36615666]
 [-0.36306682]
 [-0.35997698]
 [-0.35688714]
 [-0.3537973 ]
 [-0.35070746]
 [-0.34761762]
 [-0.34452778]
 [-0.34143794]
 [-0.3383481 ]
 [-0.33525826]
 [-0.33216843]
 [-0.32907859]
 [-0.32598875]
 [-0.32289891]
 [-0.31980907]
 [-0.31671923]
 [-0.31362939]
 [-0.31053955]
 [-0.30744971]
 [-0.30435987]
 [-0.30127003]
 [-0.29818019]
 [-0.29509035]
 [-0.29200051]
 [-0.28891067]
 [-0.28582084]
 [-0.282731  ]
 [-0.27964116]
 [-0.27655132]
 [-0.27346148]
 [-0.27037164]
 [-0.2672818 ]
 [-0.26419196]
 [-0.26110212]
 [-0.25801228]
 [-0.25492244]
 [-0.2518326 ]
 [-0.24874276]
 [-0.24565292]
 [-0.24256308]
 [-0.23947325]
 [-0.23638341]
 [-0.2332935

ValueError:  did not return arithmetic sequence: [-0.3785160151819251, -0.3723363365142123, -0.3661566578464994, -0.35997697917878657, -0.3568871398449301, -0.34761762184336087, -0.3414379431756479, -0.33525826450793517, -0.3290785858402222, -0.32289890717250946, -0.3167192285047966, -0.31053954983708376, -0.3043598711693709, -0.29818019250165806, -0.2920005138339452, -0.28582083516623213, -0.2796411564985195, -0.27346147783080665, -0.2672817991630938, -0.26110212049538095, -0.2549224418276681, -0.24874276315995525, -0.2425630844922424, -0.2363834058245292, -0.2302037271568167, -0.22711388782295994, -0.217844369821391, -0.21166469115367814, -0.2054850124859653, -0.19930533381825244, -0.1931256551505396, -0.18694597648282674, -0.18076629781511389, -0.17458661914740103, -0.1684069404796877, -0.16222726181197533, -0.15604758314426248, -0.14986790447654963, -0.14368822580883625, -0.13750854714112393, -0.13132886847341108, -0.1282390291395541, -0.11896951113798479, -0.11278983247027252, -0.

In [ ]:
X, y = gen_data(500, 40, 1)
test_loss1, loss_grid1, iters_grid1, times_grid1, test_loss_uncoded1, uncoded_times1 = run(X,y, "linear_data.csv")

In [ ]:

data = arff.loadarff('/Users/willem/Downloads/speeddating.arff')
df = pd.DataFrame(data[0])

hill_train_x, hill_test_x, hill_train_y, hill_test_y = clean_and_scale(df, "match")
hill_train_x, hill_train_y = pad(hill_train_x, hill_train_y, 7)
hill_test_x, hill_test_y = pad(hill_test_x, hill_test_y, 7)

print(hill_train_x.shape, hill_train_y.shape, hill_test_x.shape, hill_test_y.shape)

X = np.vstack((hill_test_x,hill_train_x))
y = np.vstack((hill_test_y, hill_train_y))
test_loss_real, loss_grid_real, iters_grid_real, times_grid_real, test_loss_uncoded_real, uncoded_times_real  = run(X,y, "real_data.csv")


In [ ]:
from gen_data import gen_seperable_data
X, y, w = gen_seperable_data(500, 40)

test_loss, loss_grid, iters_grid, times_grid, test_loss_uncoded, uncoded_times = run(X,y, "seperable_data.csv")

In [ ]:
X, y = gen_nonlinear_data(2000, 200, 1)

test_loss, loss_grid, iters_grid, times_grid, test_loss_uncoded, uncoded_times = run(X,y, "hdnonlinear_data.csv")

In [ ]:
def plot_3d_bar(data):

    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    rows, cols = data.shape
    x, y = np.meshgrid(np.arange(cols), np.arange(rows))

    x = x.flatten()
    y = y.flatten()
    z = np.zeros_like(x)

    dz = data.flatten()

    ax.bar3d(x, y, z, 1, 1, dz, shade=True)

    ax.set_xlabel('w_lvl')
    ax.set_ylabel('grd_lvl')
    ax.set_zlabel('z')

    plt.show()

In [ ]:
def plot_time_per_query(df):
    import numpy as np

    df_numeric = df.select_dtypes(include=[np.number])

    grouped = df_numeric.groupby(['w-quantization', 'grd-quantization']).mean()

    # Reset the index to make plotting easier
    grouped = grouped.reset_index()

    import matplotlib.pyplot as plt
    from mpl_toolkits.mplot3d import Axes3D

    # Setting up the figure and 3D axis
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')

    # Data for the bars
    x = grouped['w-quantization']
    y = grouped['grd-quantization']
    z = grouped['time']  # or any other column you wish to visualize

    # The bar positions and width
    dx = dy = 0.5
    dz = z

    ax.bar3d(x, y, [0]*len(z), dx, dy, dz, color='b')

    # Labeling the axes
    ax.set_xlabel('w-quantization')
    ax.set_ylabel('grd-quantization')
    ax.set_zlabel('Average Time Per Query')

    # Show the plot
    plt.show()

In [ ]:
file_path = 'access_measurements.csv'

column_names = [
    'w-quantization', 'grd-quantization', 'imputation', 'access',
    'query type', 'time', 'stop cond', 'iters'
]

df_nonlinear = pd.read_csv(file_path, header=None, names=column_names)

print(df_nonlinear.head())

In [7]:
bad_w = [-0.08605444406989449, -0.06381547465695116, -0.04157650524400783, -0.019337535831064498, 0.0029014335818787276, 0.02514040299482216, 0.05849885711423698, 0.08073782652718031
         , 0.09185731123365215, 0.1140962806465955, 0.1363352500595388, 0.15857421947248213, 0.18081318888542547, 0.2030521582983688, 0.2252911277113121, 0.24753009712425544]

for i in range(1,len(bad_w)):
    print(f"\n {bad_w[i] - bad_w[i-1]}")






 0.02223896941294333

 0.02223896941294333

 0.02223896941294333

 0.022238969412943226

 0.022238969412943434

 0.03335845411941482

 0.02223896941294333

 0.011119484706471838

 0.022238969412943344

 0.022238969412943316

 0.022238969412943316

 0.022238969412943344

 0.022238969412943344

 0.022238969412943288

 0.022238969412943344
